In [1]:
# CNN Tensor flow Example
# This is code is following 'https://github.com/minsuk-heo/deeplearning/blob/master/src/CNN_Tensorflow.ipynb'
import tensorflow as tf
# load data from keras
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# the last 10000 dataset set aside for validation 
x_val = x_train[50000:60000]
y_val = y_train[50000:60000]
# total 50000 data sets are used for training
x_train = x_train[0:50000]
y_train = y_train[0:50000]

/home/hanjun/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Reshape
import numpy as np

x_train = np.reshape(x_train, (50000, 28, 28, 1))
x_val = np.reshape(x_val, (10000, 28, 28, 1))
x_test = np.reshape(x_test, (10000, 28, 28, 1))


In [3]:
# Normalize

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255

x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale


In [4]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [6]:
# Define initial value generating function
def weighted_variable(shape) :
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape) :
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [7]:
# Define filter function
def conv2d(x, W) :
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# Define pooling function
def max_pool_2x2(x) :
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [8]:
# Define first convolution layer. 5 x 5 x 1 x 16 Input was 5 by 5 and gray scale and apply 16filters 
W_conv1 = weighted_variable([5,5,1,10])
b_conv1 = bias_variable([10])
# Define activation function. We use relu function
h_conv1 = tf.nn.relu( conv2d(x, W_conv1) + b_conv1)
# pooling layer
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
# Define second convolution layer. 5 x 5 x 16 x 32, 32 filters applied
W_conv2 = weighted_variable([5, 5, 10, 20])
b_conv2 = bias_variable([20])
# Define activation function, we use relu function
h_conv2 = tf.nn.relu( conv2d(h_pool1, W_conv2) + b_conv2)
# pooling layer
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
# Define fully connected layer

# First hidden layer with 128 nodes 
W_fc1 = weighted_variable([7*7*20, 128])
b_fc1 = bias_variable([128])
# Flatting h_pool1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*20])
h_fc1 = tf.nn.relu( tf.matmul( h_pool2_flat, W_fc1) + b_fc1 )

In [11]:
# second hidden layer with 10 nodes
W_fc2 = weighted_variable([128, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.relu( tf.matmul(h_fc1, W_fc2) + b_fc2)

In [12]:
#Loss function : cross-entropy
cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv ) )

In [13]:
# Optimizer : Adam
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [14]:
# Define accuracy and correctness
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Train and Test
init = tf.global_variables_initializer()

epoch_cnt = 3
batch_size = 500
iteration = len(x_train) // batch_size

with tf.Session() as sess :
    tf.set_random_seed(7777)
    sess.run(init)
    
    for epoch in range(epoch_cnt) :
        avg_loss = 0
        start = 0
        end = batch_size
        
        for i in range(iteration):
            if i%10 == 0:
                train_accuracy = accuracy.eval(feed_dict={x:x_train[start: end], y_: y_train[start: end]})
                print("step "+ str(i) + ": training accuracy: "+str(train_accuracy))
            train_step.run(feed_dict={x:x_train[start: end], y_: y_train[start: end]})
            start += batch_size; end += batch_size    
        
        # Validate model
        val_accuracy = accuracy.eval(feed_dict={x:x_val, y_: y_val})
        print("validation accuracy: "+str(val_accuracy))
        
    test_accuracy = accuracy.eval(feed_dict={x:x_test, y_: y_test}) 
    print("test accuracy: "+str(test_accuracy))

step 0: training accuracy: 0.05
step 10: training accuracy: 0.514
step 20: training accuracy: 0.58
step 30: training accuracy: 0.628
step 40: training accuracy: 0.608
step 50: training accuracy: 0.628
step 60: training accuracy: 0.636
step 70: training accuracy: 0.632
step 80: training accuracy: 0.662
step 90: training accuracy: 0.644
validation accuracy: 0.6653
step 0: training accuracy: 0.632
step 10: training accuracy: 0.67
step 20: training accuracy: 0.638


In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(x.shape)
print(y_.shape)